In [ ]:
# Self Play Experiment

In [1]:
from MCTS import MCTS
from TSP import TSPGame
from NETS import RecurrentNN, create_training_set, ConvolutionalNN, GraphConvolutionalNN
from EVALUATE import evaluation_run, plot_comparison, create_comparison
from SELFPLAY import Coach
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
class dotdict(dict):
    def __getattr__(self, name):
        return self[name]

In [3]:
num_node = 4
numAssess = 30
numEps = 300
validation_split = 0.2
numGens = 5
patience = 100

args = dotdict({
    # MCTS args
    'numMCTSSims': 100,        # Number of games moves for MCTS to simulate in self-play
    'num_node': num_node,              # Number of nodes in the graph (game)
    'cpuct': 1,
    
    # Self Play
    'numEps': numEps,
    'numAssess': numAssess,
    'numGens': numGens,
    'winThresh': 1.0           # 1 Thresh means replace on any total improvement

    })

rnn_args = dotdict({
    # State representation
    'history': True,           # Whether state representation should be a history of states - used for lstm
    'history_length': None,     # If None full history
    
    # NN args
    'lr': 0.001,
    'dropout': 0.3,
    'epochs': 20,
    'batch_size': 64,
    'validation_split': validation_split,
    'patience': patience
    })

cnn_args = dotdict({
    # State representation
    'history': False,           # Whether state representation should be a history of states - used for lstm
    
    # NN args
    'lr': 0.001,
    'dropout': 0.3,
    'epochs': 100,
    'batch_size': 64,
    'num_channels': 512,
    'validation_split': validation_split,
    'patience': patience
    })

gnn_args = dotdict({
    # State representation
    'history': False,           # Whether state representation should be a history of states - used for lstm
    
    # NN args
    'lr': 0.001,
    'epochs': 100,
    'batch_size': 64,
    'validation_split': validation_split,
    'patience': patience
    })

gnn_args.update(args)
cnn_args.update(args)
rnn_args.update(args)

In [4]:
%%time

# Create a dataset from 10000 games
dataset = create_training_set(gnn_args, 100)

CPU times: user 26.1 ms, sys: 2.23 ms, total: 28.3 ms
Wall time: 26.4 ms


In [5]:
# Create RNN
rnn = RecurrentNN(rnn_args)
gnn = GraphConvolutionalNN(gnn_args)
cnn = ConvolutionalNN(cnn_args)

In [6]:
# Train

In [7]:
%%time
gnn_hist = gnn.train(dataset)

/Users/TZilkha/anaconda3/lib/python3.6/site-packages/torch/nn/modules/loss.py:431: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/TZilkha/anaconda3/lib/python3.6/site-packages/torch/nn/modules/loss.py:431: UserWarning: Using a target size (torch.Size([4])) that is different to the input size (torch.Size([4, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


KeyboardInterrupt: 

In [8]:
# Plot training & validation v loss values
plt.plot(gnn_hist['v_loss'])
plt.plot(gnn_hist['val_v_loss'])
plt.title('v Training Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['v', 'val_v'])
plt.show()

# Plot training & validation pi loss values
plt.plot(gnn_hist['pi_loss'])
plt.plot(gnn_hist['val_pi_loss'])
plt.title('pi Training Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['pi', 'val_pi'])
plt.show()

NameError: name 'gnn_hist' is not defined

In [10]:
gnn_hist

{'loss': [1.8095494443406615,
  1.2707868123768633,
  1.2234252714590814,
  1.190780996549265,
  1.1674777968701338,
  1.1148690982227867,
  1.1572553388121445,
  1.1233019754666576,
  1.0867860963357932,
  1.0958481497237502,
  1.1016012620965874,
  1.059663784993022,
  1.0390458519885244,
  1.044280576216837,
  1.0331900517476795,
  1.062949737247817,
  1.0523211044393952,
  1.0311976404073246,
  1.048448559302411,
  1.0715057953047478,
  1.0288801999987738,
  1.0641067436077747,
  1.0697892265101105,
  1.010538234669699,
  1.0477129700037158,
  1.0042283495593352,
  1.001442225468988,
  0.9644212589950949,
  1.0172180141800498,
  0.9910185035955096,
  0.9709134002849331,
  0.9614317444981261,
  1.019462886189762,
  1.0179275354621553,
  0.9113780526586247,
  0.9736265569069188,
  0.9409506754722033,
  0.9662936040072753,
  0.9625528801249262,
  0.9192455404983985,
  0.931266726906982,
  0.9371931736623651,
  0.9489020507833873,
  0.9251317181532959,
  0.9439842245170446,
  0.9339568

In [6]:
c = Coach(cnn, rnn_args)

In [7]:
c.arena()

Saved model to disk
Assessing old...
Dataset Size 900
Training...
Train on 720 samples, validate on 180 samples
Epoch 1/100
720/720 [==============================] - 6s 8ms/step - loss: 2.2007 - pi_loss: 0.8596 - v_loss: 1.3411 - val_loss: 1.6418 - val_pi_loss: 1.1272 - val_v_loss: 0.5146
Epoch 2/100
720/720 [==============================] - 3s 4ms/step - loss: 1.1342 - pi_loss: 0.6031 - v_loss: 0.5311 - val_loss: 1.5552 - val_pi_loss: 0.9936 - val_v_loss: 0.5616
Epoch 3/100
720/720 [==============================] - 3s 4ms/step - loss: 1.1755 - pi_loss: 0.6161 - v_loss: 0.5595 - val_loss: 1.5464 - val_pi_loss: 0.9847 - val_v_loss: 0.5616
Epoch 4/100
720/720 [==============================] - 3s 4ms/step - loss: 1.1375 - pi_loss: 0.5826 - v_loss: 0.5549 - val_loss: 1.5545 - val_pi_loss: 0.9929 - val_v_loss: 0.5616
Epoch 5/100
720/720 [==============================] - 3s 4ms/step - loss: 1.1388 - pi_loss: 0.5696 - v_loss: 0.5692 - val_loss: 1.6270 - val_pi_loss: 1.0653 - val_v_loss: 

Epoch 46/100
720/720 [==============================] - 8s 11ms/step - loss: 0.6903 - pi_loss: 0.3676 - v_loss: 0.3227 - val_loss: 0.9505 - val_pi_loss: 0.6111 - val_v_loss: 0.3395
Epoch 47/100
720/720 [==============================] - 6s 8ms/step - loss: 0.7109 - pi_loss: 0.3835 - v_loss: 0.3275 - val_loss: 0.9797 - val_pi_loss: 0.6422 - val_v_loss: 0.3375
Epoch 48/100
720/720 [==============================] - 5s 7ms/step - loss: 0.6858 - pi_loss: 0.3611 - v_loss: 0.3247 - val_loss: 0.9404 - val_pi_loss: 0.6022 - val_v_loss: 0.3382
Epoch 49/100
720/720 [==============================] - 7s 10ms/step - loss: 0.6773 - pi_loss: 0.3575 - v_loss: 0.3198 - val_loss: 0.8985 - val_pi_loss: 0.5598 - val_v_loss: 0.3387
Epoch 50/100
720/720 [==============================] - 7s 9ms/step - loss: 0.6848 - pi_loss: 0.3601 - v_loss: 0.3247 - val_loss: 0.9030 - val_pi_loss: 0.5630 - val_v_loss: 0.3399
Epoch 51/100
720/720 [==============================] - 7s 10ms/step - loss: 0.6827 - pi_loss: 0.3

720/720 [==============================] - 4s 6ms/step - loss: 0.5102 - pi_loss: 0.3187 - v_loss: 0.1915 - val_loss: 0.7042 - val_pi_loss: 0.4735 - val_v_loss: 0.2307
Epoch 92/100
720/720 [==============================] - 4s 5ms/step - loss: 0.5004 - pi_loss: 0.3173 - v_loss: 0.1831 - val_loss: 0.6889 - val_pi_loss: 0.4806 - val_v_loss: 0.2083
Epoch 93/100
720/720 [==============================] - 4s 5ms/step - loss: 0.5247 - pi_loss: 0.3312 - v_loss: 0.1936 - val_loss: 0.7305 - val_pi_loss: 0.4802 - val_v_loss: 0.2503
Epoch 94/100
720/720 [==============================] - 4s 5ms/step - loss: 0.5282 - pi_loss: 0.3283 - v_loss: 0.1999 - val_loss: 0.7479 - val_pi_loss: 0.4846 - val_v_loss: 0.2633
Epoch 95/100
720/720 [==============================] - 4s 5ms/step - loss: 0.5239 - pi_loss: 0.3272 - v_loss: 0.1967 - val_loss: 0.7361 - val_pi_loss: 0.4815 - val_v_loss: 0.2547
Epoch 96/100
720/720 [==============================] - 4s 6ms/step - loss: 0.5045 - pi_loss: 0.3198 - v_loss: 0.

720/720 [==============================] - 4s 6ms/step - loss: 0.3052 - pi_loss: 0.2024 - v_loss: 0.1028 - val_loss: 0.3702 - val_pi_loss: 0.2788 - val_v_loss: 0.0914
Epoch 35/100
720/720 [==============================] - 4s 6ms/step - loss: 0.3317 - pi_loss: 0.2158 - v_loss: 0.1159 - val_loss: 0.3698 - val_pi_loss: 0.2800 - val_v_loss: 0.0899
Epoch 36/100
720/720 [==============================] - 4s 6ms/step - loss: 0.3076 - pi_loss: 0.2151 - v_loss: 0.0925 - val_loss: 0.4096 - val_pi_loss: 0.3176 - val_v_loss: 0.0920
Epoch 37/100
720/720 [==============================] - 5s 7ms/step - loss: 0.3002 - pi_loss: 0.2076 - v_loss: 0.0926 - val_loss: 0.4024 - val_pi_loss: 0.3165 - val_v_loss: 0.0860
Epoch 38/100
720/720 [==============================] - 4s 6ms/step - loss: 0.2736 - pi_loss: 0.1996 - v_loss: 0.0740 - val_loss: 0.4850 - val_pi_loss: 0.3066 - val_v_loss: 0.1784
Epoch 39/100
720/720 [==============================] - 4s 5ms/step - loss: 0.3113 - pi_loss: 0.2080 - v_loss: 0.

720/720 [==============================] - 4s 5ms/step - loss: 0.2466 - pi_loss: 0.1901 - v_loss: 0.0565 - val_loss: 0.3857 - val_pi_loss: 0.3056 - val_v_loss: 0.0801
Epoch 80/100
720/720 [==============================] - 4s 5ms/step - loss: 0.2638 - pi_loss: 0.1948 - v_loss: 0.0690 - val_loss: 0.3918 - val_pi_loss: 0.2931 - val_v_loss: 0.0987
Epoch 81/100
720/720 [==============================] - 4s 5ms/step - loss: 0.2891 - pi_loss: 0.1991 - v_loss: 0.0900 - val_loss: 0.4046 - val_pi_loss: 0.3164 - val_v_loss: 0.0882
Epoch 82/100
720/720 [==============================] - 4s 5ms/step - loss: 0.2899 - pi_loss: 0.1969 - v_loss: 0.0930 - val_loss: 0.3951 - val_pi_loss: 0.3105 - val_v_loss: 0.0847
Epoch 83/100
720/720 [==============================] - 4s 5ms/step - loss: 0.2802 - pi_loss: 0.1901 - v_loss: 0.0901 - val_loss: 0.4003 - val_pi_loss: 0.3110 - val_v_loss: 0.0892
Epoch 84/100
720/720 [==============================] - 4s 5ms/step - loss: 0.2716 - pi_loss: 0.1857 - v_loss: 0.

KeyboardInterrupt: 

In [23]:
rnn.load_model('Generation_1')

Loaded model from disk


In [26]:
rnn.train(dataset)

Training...
Train on 240 samples, validate on 60 samples
Epoch 1/20
240/240 [==============================] - 4s 17ms/step - loss: 1.9883 - pi_loss: 1.4030 - v_loss: 0.5853 - val_loss: 1.8350 - val_pi_loss: 1.2948 - val_v_loss: 0.5401
Epoch 2/20
240/240 [==============================] - 0s 668us/step - loss: 1.8370 - pi_loss: 1.2518 - v_loss: 0.5853 - val_loss: 1.7067 - val_pi_loss: 1.1666 - val_v_loss: 0.5401
Epoch 3/20
240/240 [==============================] - 0s 656us/step - loss: 1.7253 - pi_loss: 1.1400 - v_loss: 0.5853 - val_loss: 1.6573 - val_pi_loss: 1.1172 - val_v_loss: 0.5401
Epoch 4/20
240/240 [==============================] - 0s 660us/step - loss: 1.7005 - pi_loss: 1.1153 - v_loss: 0.5853 - val_loss: 1.6542 - val_pi_loss: 1.1141 - val_v_loss: 0.5401
Epoch 5/20
240/240 [==============================] - 0s 650us/step - loss: 1.6850 - pi_loss: 1.0997 - v_loss: 0.5853 - val_loss: 1.6249 - val_pi_loss: 1.0847 - val_v_loss: 0.5401
Epoch 6/20
240/240 [========================

In [25]:
rnn.create_net()

In [20]:
rnn.train(dataset)

Training...
Train on 240 samples, validate on 60 samples
Epoch 1/20
240/240 [==============================] - 4s 15ms/step - loss: 2.7657 - pi_loss: 2.1804 - v_loss: 0.5853 - val_loss: 2.5057 - val_pi_loss: 1.9656 - val_v_loss: 0.5401
Epoch 2/20
240/240 [==============================] - 0s 614us/step - loss: 2.4479 - pi_loss: 1.8626 - v_loss: 0.5853 - val_loss: 2.2166 - val_pi_loss: 1.6765 - val_v_loss: 0.5401
Epoch 3/20
240/240 [==============================] - 0s 617us/step - loss: 2.1761 - pi_loss: 1.5909 - v_loss: 0.5853 - val_loss: 1.9763 - val_pi_loss: 1.4362 - val_v_loss: 0.5401
Epoch 4/20
240/240 [==============================] - 0s 657us/step - loss: 1.9523 - pi_loss: 1.3670 - v_loss: 0.5853 - val_loss: 1.7794 - val_pi_loss: 1.2393 - val_v_loss: 0.5401
Epoch 5/20
240/240 [==============================] - 0s 610us/step - loss: 1.7840 - pi_loss: 1.1987 - v_loss: 0.5853 - val_loss: 1.6765 - val_pi_loss: 1.1364 - val_v_loss: 0.5401
Epoch 6/20
240/240 [========================